In [1]:
import pprint
import time

import zmq
from sympy.physics.units import current

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
socket = ctx.socket(zmq.REQ)

ip = 'localhost'  # If you talk to a different machine use its IP.
port = 50020  # The port defaults to 50020. Set in Pupil Capture GUI.

socket.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
socket.send_string('SUB_PORT')
sub_port = socket.recv_string()

# Request 'PUB_PORT' for writing data
socket.send_string('PUB_PORT')
pub_port = socket.recv_string()

socket.close()

In [2]:
import msgpack

socket = ctx.socket(zmq.REQ)
socket.connect(f'tcp://{ip}:{port}')

# Define the new configuration for fixation parameters
notification = {
    "subject": "start_plugin",
    "name": "Fixation_Detector",
    "args": {
        "max_dispersion": 5,
        "min_duration": 600,
    },
}

topic = 'notify.' + notification['subject']
payload = msgpack.dumps(notification)
socket.send_string(topic, flags=zmq.SNDMORE)
socket.send(payload)
socket.close()

In [3]:
import requests

# Define the URL of the lamp
url = "http://10.2.2.33:1880/r402/theglobe"

# Define the payload to turn the lamp off
payload = {
    "on": True,  # Turn the lamp off
    "brightness": 100
}

# Send the PUT request to change the state of the lamp
try:
    response = requests.put(url, json=payload)
    print(response)
    if response.status_code == 200:
        print("The lamp has been turned on successfully.")
    else:
        print(f"Failed to turn off the lamp. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    
# Sent a GET request to retrieve the Thing Description
response = requests.get(url)
print(response)

<Response [200]>
The lamp has been turned on successfully.
<Response [200]>


In [4]:
# Setup ZeroMQ context and subscriber socket
ctx = zmq.Context()

def create_socket(ctx_c, ip_c, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{sub_port}')
    for topic in topics:
        sub.subscribe(topic)
    return sub

In [8]:
import numpy as np
import cv2
import msgpack


def receive_data(sock):
    """Receive data from the socket and return the topic and payload."""
    data = sock.recv_multipart()
    t = data[0].decode("utf-8")
    p = msgpack.unpackb(data[1], raw=False) if t == 'fixations' else data[2]
    return t, p

def decode_frame(frame_data):
    """Decode frame data from socket payload."""
    return cv2.imdecode(np.frombuffer(frame_data, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_object_in_gaze(gaze, r):
    for result in r:
        boxes = result.boxes
        classes = boxes.cls
        for i, box in enumerate(boxes.xyxyn):
            # Check if the gaze point lies within the bounding box of detected objects
            if box[0] <= gaze[0] <= box[2] and box[1] <= (1 - gaze[1]) <= box[3]:
                return result.names[int(classes[i])]
    return None

def detect_camera_movement(frame1, frame2, tr=None):
    # Convert frames to grayscale
    prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Parameters for optical flow (Lucas-Kanade method)
    lk_params = dict(winSize=(21, 21), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

    # Detect good features to track in the first frame
    prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7))

    # Calculate optical flow (track points from prev_frame to curr_frame)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_points, None, **lk_params)

    # Select good points (where the flow has been successfully found)
    good_prev_points = prev_points[status == 1]
    good_next_points = next_points[status == 1]

    # Calculate the displacement of points
    displacements = good_next_points - good_prev_points
    avg_displacement = displacements.mean(axis=0)

    # Direction based on average displacement
    dx, dy = avg_displacement
    
    if tr:
        if abs(dx) > tr or abs(dy) > tr:
            return True
        return False
    
    tr_v = 30
    tr_h = 30
    if abs(dx) > abs(dy):  # Horizontal movement
        if dx > tr_h:
            return f"Left"
        elif dx < -1 * tr_h:
            return f"Right"
    else:  # Vertical movement
        if dy > tr_v:
            return f"Up"
        elif dy < -1 * tr_v:
            return f"Down"

    return None

def process_gesture(head_direction, state):
    """Update gesture state and determine if a gesture is completed."""
    interaction = None
    
    if head_direction == 'Up' and state is None:
        state = 'Up'
    elif head_direction is None and state == 'Up':
        interaction = "Increase gesture detected (up followed by down)"
        state = None
    elif head_direction == 'Down' and state is None:
        state = 'Down'
    elif head_direction is None and state == 'Down':
        interaction = "Decrease gesture detected (down followed by up)"
        state = None
    elif head_direction == 'Left' and state is None:
        state = 'Left'
    elif head_direction is None and state == 'Left':
        interaction = "Left gesture detected"
        state = None
    elif head_direction == 'Right' and state is None:
        state = 'Right'
    elif head_direction is None and state == 'Right':
        interaction = "Right gesture detected"
        state = None
    
    return interaction, state



In [9]:
import requests

def set_lamp_brightness(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    
    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)
        
        # get the current brightness of the lamp
        curr_brighness = response.json()["brightness"]
        lamp_brighness = curr_brighness
        
        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "Increase":
            if curr_brighness <= 80:
                lamp_brighness += 20
            else:
                lamp_brighness = 100
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "Decrease":
            if curr_brighness >= 20:
                lamp_brighness -= 20
            else:
                lamp_brighness = 0
    
        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "brightness": lamp_brighness
            }
            
            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp brightness has been set to {lamp_brighness}.")
                else:
                    print(f"Failed to turn off the lamp. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        
def set_lamp_color(mode):
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    colors = ["yellow", "red", "green", "blue", "magenta"]
    
    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)
        
        # get the current color of the lamp
        curr_color = response.json()["color"]
        if curr_color in colors:
            curr_index = colors.index(curr_color)
        else:
            curr_index = 0
        
        # if mode is increase, increase the brightness by 10 if it is less than 100
        if mode == "Left":
            if curr_index == 0:
                curr_index = len(colors) - 1
            else:
                curr_index = curr_index - 1
        # if mode is decrease, decrease the brightness by 10 if it is greater than 0
        elif mode == "Right":
            if curr_index == len(colors) - 1:
                curr_index = 0
            else:
                curr_index = curr_index + 1
    
        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "color": colors[curr_index]
            }
            
            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp color has been set to {colors[curr_index]}.")
                else:
                    print(f"Failed to set lamp color. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [13]:
from time import sleep
from ultralytics import YOLO
from zmq_tools import *
import time

ctx = zmq.Context()
requester = ctx.socket(zmq.REQ)
requester.connect('tcp://localhost:50020') #change ip if using remote machine

# request 'SUB_PORT' for reading data
requester.send_string('SUB_PORT')
ipc_sub_port = requester.recv_string()

model = YOLO("../.local/models/object_detection/yolov11n_trained.pt", verbose=False) 

last_frame = None
obj_detected = None

fixation_detected = False
movement_state = None
while not fixation_detected:
    print("Waiting for fixation")
    socket_fixations = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('fixations',))
    _, payload = socket_fixations.recv()
    gaze_position = payload["norm_pos"]
    print("Fixation detected")
    
    print("Waiting for frame")
    socket_frames = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',))
    _, payload = socket_frames.recv()
    curr_frame = decode_frame(payload['__raw_data__'][0])
    print("Frame received")
    
    if last_frame is None and obj_detected is None:
        last_frame = curr_frame
        obj_detected = model(last_frame)
    
    print("Checking for camera movement")
    image_shift = detect_camera_movement(last_frame, curr_frame, tr=5)
    if image_shift:
        print("Camera movement detected")
        obj_detected = model(curr_frame)
        last_frame = curr_frame
    
    print("Checking for object in gaze")
    obj = get_object_in_gaze(gaze_position, obj_detected)
    if obj and "HueLamp" in obj:
        print("Object detected in gaze, starting gesture detection")
        gesture_detected = False
        while not gesture_detected:
            socket_frames = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',))
            _, payload = socket_frames.recv()
            frame = decode_frame(payload['__raw_data__'][0])
            direction = detect_camera_movement(curr_frame, frame)
                
            gesture, movement_state = process_gesture(direction, movement_state)
            if gesture:
                if "Increase" in gesture:
                    print(gesture)
                    set_lamp_brightness("Increase")
                elif "Decrease" in gesture:
                    print(gesture)
                    set_lamp_brightness("Decrease")
                elif "Left" in gesture:
                    set_lamp_color("Left")
                    print(gesture)
                elif "Right" in gesture:
                    set_lamp_color("Right")
                    print(gesture)
                gesture_detected = True
                time.sleep(0.5)
            

Waiting for fixation
Fixation detected
Waiting for frame
Frame received

0: 384x640 1 HueLampWhiteHalfRound, 17.7ms
Speed: 3.4ms preprocess, 17.7ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Checking for camera movement
Checking for object in gaze
Waiting for fixation
Fixation detected
Waiting for frame
Frame received
Checking for camera movement
Camera movement detected

0: 384x640 1 HueLampYellow, 17.4ms
Speed: 2.4ms preprocess, 17.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Checking for object in gaze
Object detected in gaze, starting gesture detection
Lamp color has been set to red.
Right gesture detected
Waiting for fixation
Fixation detected
Waiting for frame
Frame received
Checking for camera movement
Camera movement detected

0: 384x640 1 HueLampRed, 16.8ms
Speed: 2.1ms preprocess, 16.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
Checking for object in gaze
Object detected in gaze, starting gesture detection



KeyboardInterrupt



In [ ]:
import cv2

def frame_change(frame1, frame2, threshold=200):
    # Convert frames to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Detect and compute keypoints and descriptors
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Create a Brute-Force Matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(des1, des2)

    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    print(len(matches))

    # If matches are below the threshold, images are completely different
    return len(matches) < threshold

In [ ]:
socket_frames = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',))
_, payload = socket_frames.recv()
curr_frame = decode_frame(payload['__raw_data__'][0])

while True:
    time.sleep(1)
    socket_frames = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',))
    _, payload = socket_frames.recv()
    frame = decode_frame(payload['__raw_data__'][0])
    direction = detect_camera_movement(curr_frame, frame)
    print(direction)
    curr_frame = frame
                    

In [ ]:
from zmq_tools import *
import time

ctx = zmq.Context()
requester = ctx.socket(zmq.REQ)
requester.connect('tcp://localhost:50020') #change ip if using remote machine

# request 'SUB_PORT' for reading data
requester.send_string('SUB_PORT')
ipc_sub_port = requester.recv_string()

monitor = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',)) #change ip if using remote machine

s1 = monitor.recv()[1]
print(s1)

In [ ]:
from time import sleep
import pprint
from zmq_tools import *
import matplotlib.pyplot as plt
ctx = zmq.Context()
requester = ctx.socket(zmq.REQ)
requester.connect('tcp://localhost:50020') #change ip if using remote machine

# request 'SUB_PORT' for reading data
requester.send_string('SUB_PORT')
ipc_sub_port = requester.recv_string()

while not fixation_detected:
    socket_fixations = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('fixations',))
    _, payload = socket_fixations.recv()
    print("Fixation detected")
    gaze_position = payload["norm_pos"]
    pprint.pprint(payload)
    
    socket_frames = Msg_Receiver(ctx,'tcp://localhost:%s'%ipc_sub_port,topics=('frame.world',))
    _, payload = socket_frames.recv()
    curr_frame = decode_frame(payload['__raw_data__'][0])
    
    # plot the fixation point on the frame
    x, y = int(gaze_position[0] * curr_frame.shape[1]), int((1 - gaze_position[1]) * curr_frame.shape[0])
    cv2.circle(curr_frame, (x, y), 10, (255, 0, 0), -1)
    plt.imshow(curr_frame, cmap='gray')
    break
    
                
                    
            